In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import numpy as np
import os
import cv2
from tqdm import tqdm

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import tensorflow as tf

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
DATADIR = r"/content/gdrive/My Drive/Weather"

CATEGORIES = ['Cloudy', 'Rain', 'sunny', 'sunset']

IMG_SIZE =150
img_rows, img_cols = 150,150
training_data = []

def create_training_data():
    for category in CATEGORIES:
        
        path = os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)
        
        for img in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num]) 
            except Exception as e:
                pass

create_training_data()

print(len(training_data))

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

100%|██████████| 358/358 [01:21<00:00,  4.39it/s]

1123


In [ ]:
X1 = np.array(X)
X1.shape

(1123, 150, 150, 3)

In [ ]:
y = np.array(y)
y.shape

(1123,)

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=4)
X_train = X_train.reshape(X_train.shape[0],img_rows, img_cols,3)
X_test = X_test.reshape(X_test.shape[0],img_rows, img_cols,3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(y_train, 4)
Y_test = np_utils.to_categorical(y_test, 4)

In [ ]:
X_test.shape

(225, 150, 150, 3)

In [ ]:
Y_train.shape

(898, 4)

In [ ]:
input_shape = [150, 150, 3]
model = Sequential()
model.add(ResNet101(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=4,
    
))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dense(units=4,activation="softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Functional)       (None, 5, 5, 2048)        42658176  
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              52429824  
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 4100      
Total params: 95,092,100
Trainable params: 94,986,756
Non-trainable params: 105,344
_________________________________________________________________


In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(X_train,Y_train,batch_size=32,epochs=10,verbose=1,validation_split=0.2)

Epoch 1/10
23/23 [==============================] - 544s 23s/step - loss: 71.3559 - accuracy: 0.3027 - val_loss: 16.3585 - val_accuracy: 0.2889
Epoch 2/10
23/23 [==============================] - 525s 23s/step - loss: 1.2681 - accuracy: 0.5158 - val_loss: 1.4810 - val_accuracy: 0.2889
Epoch 3/10
23/23 [==============================] - 507s 22s/step - loss: 1.2273 - accuracy: 0.4807 - val_loss: 1.3928 - val_accuracy: 0.2389
Epoch 4/10
23/23 [==============================] - 504s 22s/step - loss: 0.9824 - accuracy: 0.5230 - val_loss: 1.4291 - val_accuracy: 0.2833
Epoch 5/10
23/23 [==============================] - 507s 22s/step - loss: 0.9014 - accuracy: 0.5587 - val_loss: 1.5084 - val_accuracy: 0.2833
Epoch 6/10
23/23 [==============================] - 508s 22s/step - loss: 0.8733 - accuracy: 0.5669 - val_loss: 1.5840 - val_accuracy: 0.2833
Epoch 7/10
23/23 [==============================] - 498s 22s/step - loss: 0.7872 - accuracy: 0.6322 - val_loss: 1.7658 - val_accuracy: 0.2389
Epoc

In [ ]:
scores = model.evaluate(X_train, Y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 30.51%


In [ ]:
model.save("/content/gdrive/My Drive/model/weather_resnet.h5")
print("Saved model to disk")

Saved model to disk
